<a href="https://colab.research.google.com/github/sindla97/Machine-Learning/blob/main/humana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [7]:
df=pd.read_csv("2022_Competition_Training.csv", low_memory=False)

In [8]:
df.head()

,id,rx_hum_19_pmpm_cost,cmsd2_can_mal_male_genital_pmpm_ct,cmsd2_inj_surgical_pmpm_ct,rev_pm_capd_pmpm_cd_ct,cci_chf_pmpm_ct,rx_phar_cat_safeway_pmpm_ct,cmsd2_ext_car_pmpm_ct,cmsd2_dig_liver_pmpm_ct,cmsd2_mus_arthropath_pmpm_ct,...,cmsd2_ext_nature_pmpm_ct,cmsd2_gus_m_genital_pmpm_ct,atlas_totalpopacs,cmsd2_neo_neo_other_pmpm_ct,cmsd2_dig_hernia_pmpm_ct,cmsd2_sns_blood_pmpm_ct,cmsd2_ext_compl_medical_care_pmpm_ct,rwjf_food_env_inx,atlas_snapspth16,hi_flag
0,100039218.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.08,33511.0,0.0,0.0,0.00,0.0,9.1,NaN,0
1,100042427.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,176797.0,0.0,0.0,0.00,0.0,6.8,0.670459,0
2,100055103.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,380673.0,0.0,0.0,0.00,0.0,8.3,0.879186,0
3,100074556.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,267696.0,0.0,0.0,0.00,0.0,7.3,0.746201,0
4,100079932.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,19095.0,0.0,0.0,0.25,0.0,6.9,NaN,0


In [9]:
df.shape

(48300, 881)

In [25]:
df_null=pd.DataFrame(df.isnull().sum()).iloc[[x for x in range(df.shape[1]) if df.isnull().sum()[x] > 0.50*(df.shape[0])],:]


<ipython-input-25-b90e7bb9022c>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_null=pd.DataFrame(df.isnull().sum()).iloc[[x for x in range(df.shape[1]) if df.isnull().sum()[x] > 0.50*(df.shape[0])],:]


In [35]:
df_null = df_null.reset_index()

In [28]:
df['hi_flag'].value_counts()

,count
hi_flag,
0,46182
1,2118


In [49]:
columns = [x for x in df.columns if x not in ['id','hi_flag', 'cons_mobplus','cms_ra_factor_type_cd', 'cons_homstat', 'sex_cd', 'lang_spoken_cd','rucc_category','cms_race_cd']+list(df_null.index)]
len(columns)

872

In [50]:
from sklearn.model_selection import *
X_train, X_test, y_train, y_test = train_test_split(
    df[columns], df['hi_flag'], test_size=0.2, random_state=42)


In [78]:
y_train.sum(), y_test.sum()

(1711, 407)

In [40]:
from xgboost import XGBClassifier

In [69]:
bst = XGBClassifier(n_estimators=1000, max_depth=7, learning_rate=0.001,reg_lambda= 100,objective='binary:logistic', reg_alpha=0.5, scale_pos_weight=46182/2118)
# fit model
bst.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.001, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [70]:
# make predictions
preds = bst.predict(X_test)
preds_train = bst.predict(X_train)


In [75]:
from sklearn.metrics import *
accuracy = accuracy_score(y_test, preds)
accuracy_train = accuracy_score(y_train, preds_train)
print(f'test_acc : {accuracy * 100.0}, train_acc:{accuracy_train*100.0}')

test_acc : 73.20910973084887, train_acc:75.09316770186335


In [76]:
confusion_matrix(y_test, preds), confusion_matrix(y_train, preds_train)

(array([[6843, 2410],
        [ 178,  229]]),
 array([[27605,  9324],
        [  300,  1411]]))

In [73]:
print(classification_report(y_train, preds_train))

              precision    recall  f1-score   support

           0       0.99      0.75      0.85     36929
           1       0.13      0.82      0.23      1711

    accuracy                           0.75     38640
   macro avg       0.56      0.79      0.54     38640
weighted avg       0.95      0.75      0.82     38640



In [107]:
df_col = pd.DataFrame(bst.feature_importances_, columns).reset_index()
df_col.sort_values(0, ascending=False,inplace=True)

In [108]:
df_col.head(10)

,index,0
399,cons_stlindex,0.052605
396,cms_orig_reas_entitle_cd,0.029852
689,total_physician_office_allowed_pmpm_cost,0.012659
532,cms_low_income_ind,0.011489
101,cons_hxmioc,0.010167
731,cmsd2_skn_radiation_pmpm_ct,0.008053
272,cms_tot_partd_payment_amt,0.007451
29,cmsd2_vco_circumstances_pmpm_ct,0.007383
786,rx_hum_08_pmpm_ct,0.006947
543,rev_pm_ct_pmpm_cd_ct,0.006554


In [101]:
df.groupby(['hi_flag','cms_orig_reas_entitle_cd'])['cms_orig_reas_entitle_cd'].count().unstack().fillna(0)

cms_orig_reas_entitle_cd,0.0,1.0,2.0,3.0
hi_flag,,,,
0,32360,12120,13,26
1,999,910,2,3
